<a href="https://colab.research.google.com/github/EleonoraBaim/NPS_Dialogue_system/blob/main/nb_%26_cat_boost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing and importing libraries

In [ ]:
!pip install pymorphy2

     |████████████████████████████████| 55 kB 3.2 MB/s 
     |████████████████████████████████| 8.2 MB 14.4 MB/s 


In [ ]:
pip install  scikit-multilearn

     |████████████████████████████████| 89 kB 5.5 MB/s 


In [ ]:
pip install catboost

     |████████████████████████████████| 76.3 MB 1.3 MB/s 


In [ ]:
import os
import json
import numpy as np
import pandas as pd
import re
import pickle

#from transformers import BertTokenizer
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import LabelPowerset


from sklearn.naive_bayes import MultinomialNB
from catboost import Pool, CatBoostClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, hamming_loss, accuracy_score
import pymorphy2

project_path = "/content/drive/MyDrive/Colab_Notebooks/BERT/"

min_df = 1/1000
max_df = 1/20
dataset_folder = "ready_datasets"
tokenizer_model_path = "BERT_models/rubert-base-cased-conversational"
model_path = 'model_parts'

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


# Data Importing and preprocessing


In [ ]:
#IMPORTING DATA
print('Importing data...', '\n')

train_df = pd.read_csv(str(project_path + dataset_folder + '/'+'train_dataset.csv'))
train_df.drop(labels = 'Unnamed: 0', axis = 1, inplace = True)

test_df = pd.read_csv(str(project_path + dataset_folder + '/'+ 'test_dataset.csv'))
test_df.drop(labels = 'Unnamed: 0', axis = 1, inplace = True)

val_df = pd.read_csv(str(project_path + dataset_folder + '/'+'val_dataset.csv'))
val_df.drop(labels = 'Unnamed: 0', axis = 1, inplace = True)

text = 'CONTEXT'
norm_text = 'normalized'
target_list = train_df.columns[(train_df.columns!='CONTEXT')&(train_df.columns!='normalized')]

Importing data... 



In [ ]:
print('Text Vectorization: ')

# Создаем векторизатор
tfidf_text = TfidfVectorizer(ngram_range=(1, 2), min_df=min_df, max_df=max_df)
tfidf_text.fit(train_df[norm_text])

print('Vocab size text: ' + str(len(tfidf_text.vocabulary_)))

with open(str(project_path + model_path +  'tfidf_vectorizer_text.pickle'), 'wb') as handle:
    pickle.dump(tfidf_text, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print('Text vectoriztor saved to ' + str(project_path + model_path +'/'+'tfidf_vectorizer_text.pickle'))

train_data = tf_idf_transform(train_df[norm_text], tfidf_text)
test_data = tf_idf_transform(test_df[norm_text], tfidf_text)
val_data = tf_idf_transform(val_df[norm_text], tfidf_text)

tfdf_cols = train_data.columns

train_data = train_data.join(train_df)
val_data = val_data.join(val_df)
test_data = test_data.join(test_df)

Text Vectorization: 
Vocab size text: 2169
Text vectoriztor saved to /content/drive/MyDrive/Colab_Notebooks/BERT/model_parts/tfidf_vectorizer_text.pickle


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
100%|██████████| 2169/2169 [00:06<00:00, 350.73it/s]
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
100%|██████████| 2169/2169 [00:02<00:00, 861.36it/s]
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
100%|██████████| 2169/2169 [00:02<00:00, 885.

# Model training


## NB

In [ ]:
from sklearn.multioutput import MultiOutputRegressor
#ML
model_NB = MultiOutputRegressor(MultinomialNB())

model_NB.fit(train_data[tfdf_cols], train_data[target_list])

preds_NB = model_NB.predict(test_data[tfdf_cols])

#print(confusion_matrix(y_validation, preds_NB))
print(classification_report(test_data[target_list], preds_NB))

              precision    recall  f1-score   support

           0       0.68      0.32      0.44       146
           1       0.85      0.41      0.55       285
           2       0.68      0.34      0.45       481
           3       0.73      0.08      0.15       132
           4       0.78      0.20      0.31       458
           5       0.96      0.25      0.39       106
           6       0.83      0.58      0.68       322
           7       0.74      0.45      0.56       525
           8       0.93      0.29      0.44       184
           9       0.81      0.26      0.40       160
          10       0.93      0.28      0.44       250
          11       0.91      0.24      0.37       123
          12       0.88      0.16      0.27       218
          13       0.89      0.23      0.36       502
          14       0.76      0.55      0.64       498
          15       0.94      0.18      0.31       181
          16       1.00      0.01      0.02       197
          17       0.33    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Cat Boost

In [ ]:
task_type = 'CPU'

model_CB = CatBoostClassifier(iterations=1000, random_state=1, learning_rate=0.05, verbose=10000,
                           od_type="Iter", od_wait=200, eval_metric='HammingLoss', use_best_model=True, depth=4, loss_function='MultiLogloss',task_type=task_type)

# Данные в модель подаются через Pool
p_train = Pool(train_data[tfdf_cols], train_data[target_list])
p_val = Pool(val_data[tfdf_cols], val_data[target_list])

model_CB.fit(p_train, eval_set=p_val)

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

0:	learn: 0.0404060	test: 0.0391652	best: 0.0391652 (0)	total: 3.78s	remaining: 1h 2m 59s
999:	learn: 0.0283918	test: 0.0282903	best: 0.0282903 (999)	total: 52m 46s	remaining: 0us

bestTest = 0.02829031745
bestIteration = 999



In [ ]:
preds_CB = model_CB.predict(test_data[tfdf_cols])

In [ ]:
print(classification_report(test_data[target_list], preds_CB))
print("Hamming Loss:", hamming_loss(test_data[target_list], preds_CB))

              precision    recall  f1-score   support

           0       0.91      0.29      0.45       146
           1       0.78      0.80      0.79       285
           2       0.75      0.18      0.29       481
           3       0.94      0.12      0.21       132
           4       0.85      0.18      0.29       458
           5       0.90      0.42      0.57       106
           6       0.89      0.42      0.57       322
           7       0.84      0.36      0.50       525
           8       0.84      0.80      0.82       184
           9       0.70      0.29      0.41       160
          10       0.68      0.70      0.69       250
          11       0.87      0.37      0.51       123
          12       0.79      0.28      0.42       218
          13       0.86      0.11      0.19       502
          14       0.83      0.30      0.44       498
          15       0.71      0.87      0.79       181
          16       1.00      0.01      0.01       197
          17       0.00    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
model_CB.save_model(str(project_path+model_path+'/'+'cat_boost.cbm'),
           format="cbm",
           export_parameters=None,
           pool=None)